# 1. Thư viện

In [1]:
import os
import pandas as pd

from dotenv import load_dotenv
from langchain.docstore.document import Document
from langchain_community.embeddings import GPT4AllEmbeddings
from langchain_experimental.text_splitter import SemanticChunker
from langchain_qdrant import QdrantVectorStore

In [2]:
load_dotenv()

True

In [3]:
URL_QDRANT = os.getenv('URL_QDRANT')
API_QDRANT = os.getenv('API_QDRANT')
COLLECTION_NAME_COMMON_DB_TINH_HUONG_TEST_CSV=os.getenv("COLLECTION_NAME_COMMON_DB_TINH_HUONG_TEST_CSV")
CONTENT_PAYLOAD_KEY=os.getenv("CONTENT_PAYLOAD_KEY")
METADATA_PAYLOAD_KEY=os.getenv("METADATA_PAYLOAD_KEY")

BREAKPOINT_THRESHOLD_TYPE = os.getenv("BREAKPOINT_THRESHOLD_TYPE")

# 2. Đọc file data

In [4]:
# Đọc dữ liệu từ file CSV và chỉ lấy các cột 'Situation' và 'Answer'
csv_file = r'data\tinh_huong\bhxh_bhyt_bhtn_data.csv'
df = pd.read_csv(csv_file, usecols=['Situation', 'Answer'])

In [5]:
display(df)

,Situation,Answer
0,"Tôi xin hỏi, một xã giai đoạn 2016-2020 là xã ...",Bộ Y tế trả lời vấn đề này như sau:Ngày 19/10/...
1,Tôi đóng BHXH từ tháng 6/2011 đến tháng 12/201...,Bảo hiểm xã hội Việt Nam trả lời vấn đề này nh...
2,"Tôi xin hỏi, trên giấy ra viện có ghi chú ngườ...",Bảo hiểm xã hội Việt Nam trả lời vấn đề này nh...
3,"Tôi năm nay 53 tuổi, đóng BHXH tự nguyện được ...",Bảo hiểm xã hội Việt Nam trả lời vấn đề này nh...
4,Tôi đóng BHXH từ tháng 1/2020 đến tháng 12/202...,Bảo hiểm xã hội Việt Nam trả lời vấn đề này nh...
...,...,...
1853,Theo quy định tại Điều 187 Bộ luật Lao động: “...,Bộ Lao động – Thương binh và Xã hội trả lời vấ...
1854,Tôi là giáo viên tại Trung tâm Kỹ thuật tổng h...,Bộ Lao động – Thương binh và Xã hội trả lời vấ...
1855,Tôi có 11 năm tham gia BHXH và được BHXH TP Hà...,BHXH Việt Nam trả lờivấn đề nàynhư sau:Theo qu...
1856,"Bố đẻ của tôi công tác tại xã, đóng BHXH từ nă...",Bảo hiểm xã hội Việt Nam trả lời vấn đề này nh...


# 3. Load 2 cột Situation và Answer vào Documents

In [6]:
# Tạo tài liệu từ các cột 'Situation' và 'Answer'
documents = []
for _, row in df.iterrows():
    situation = row['Situation']
    answer = row['Answer']
    document = f"Situation: {situation}\nAnswer: {answer}"
    documents.append(document)

In [7]:
# In ra 3 document đầu tiên
for i in range(3):
    print(f"Document {i + 1}:\n{documents[i]}\n")

Document 1:
Situation: Tôi xin hỏi, một xã giai đoạn 2016-2020 là xã khu vực III, sang giai đoạn 2021-2025 là xã khu vực I, đã đạt chuẩn xã nông thôn mới thì người dân tộc thiểu số trên địa bàn xã này có thuộc nhóm đối tượng được ngân sách Nhà nước hỗ trợ đóng BHYT không?
Answer: Bộ Y tế trả lời vấn đề này như sau:Ngày 19/10/2023, Chính phủ ban hành Nghị định số75/2023/NĐ-CPsửa đổi, bổ sung một số điều của Nghị định số 146/2023/NĐ-CP ngày 17/10/2018 của Chính phủ quy định chi tiết và hướng dẫn biện pháp thi hành một số điều của Luật BHYT, trong đó có quy định:Người dân tộc thiểu số đang sinh sống tại địa bàn các xã khu vực II, khu vực III, thôn đặc biệt khó khăn thuộc vùng đồng bào dân tộc thiểu số và miền núi giai đoạn 2016-2020 mà các xã này không còn trong danh sách các xã khu vực II, khu vực III, thôn đặc biệt khó khăn thuộc vùng đồng bào dân tộc thiểu số và miền núi giai đoạn 2021-2025 theo quyết định của Thủ tướng Chính phủ tiếp tục được ngân sách Nhà nước hỗ trợ tối thiểu 70% mứ

In [8]:
# Chuyển đổi các chuỗi thành các đối tượng Document
documents = [Document(page_content=doc) for doc in documents]

In [15]:
documents[0]

Document(page_content='Situation: Tôi xin hỏi, một xã giai đoạn 2016-2020 là xã khu vực III, sang giai đoạn 2021-2025 là xã khu vực I, đã đạt chuẩn xã nông thôn mới thì người dân tộc thiểu số trên địa bàn xã này có thuộc nhóm đối tượng được ngân sách Nhà nước hỗ trợ đóng BHYT không?\nAnswer: Bộ Y tế trả lời vấn đề này như sau:Ngày 19/10/2023, Chính phủ ban hành Nghị định số75/2023/NĐ-CPsửa đổi, bổ sung một số điều của Nghị định số 146/2023/NĐ-CP ngày 17/10/2018 của Chính phủ quy định chi tiết và hướng dẫn biện pháp thi hành một số điều của Luật BHYT, trong đó có quy định:Người dân tộc thiểu số đang sinh sống tại địa bàn các xã khu vực II, khu vực III, thôn đặc biệt khó khăn thuộc vùng đồng bào dân tộc thiểu số và miền núi giai đoạn 2016-2020 mà các xã này không còn trong danh sách các xã khu vực II, khu vực III, thôn đặc biệt khó khăn thuộc vùng đồng bào dân tộc thiểu số và miền núi giai đoạn 2021-2025 theo quyết định của Thủ tướng Chính phủ tiếp tục được ngân sách Nhà nước hỗ trợ tối 

# 4. Model Embedding

In [9]:
model_name = "all-MiniLM-L6-v2.gguf2.f16.gguf"
gpt4all_kwargs = {'allow_download': 'True'}

gpt4all_embedding = GPT4AllEmbeddings(
    model_name=model_name,
    gpt4all_kwargs=gpt4all_kwargs
    )

# 5. Semantic Chunker

In [10]:
text_splitter = SemanticChunker(
    embeddings = gpt4all_embedding, breakpoint_threshold_type = BREAKPOINT_THRESHOLD_TYPE
    )

# 6. Tạo 1 Collection mới

In [11]:
url = URL_QDRANT
api_key = API_QDRANT

# Chia các tài liệu thành các đoạn nhỏ
docs = text_splitter.split_documents(documents)

qdrant = QdrantVectorStore.from_documents(
    documents = docs,
    embedding = gpt4all_embedding,
    url=url,
    prefer_grpc=True,
    api_key=api_key,
    collection_name=COLLECTION_NAME_COMMON_DB_TINH_HUONG_TEST_CSV,
)

# 7. Sử dụng Collection đã có

In [12]:
qdrant = QdrantVectorStore.from_existing_collection(
    embedding = gpt4all_embedding,
    collection_name=COLLECTION_NAME_COMMON_DB_TINH_HUONG_TEST_CSV,
    url = URL_QDRANT,
    api_key= API_QDRANT,
)

In [19]:
query = """Tôi đóng BHXH từ tháng 6/2011 đến tháng 12/2014, tháng 8/2014 tôi sinh con. Do công ty nợ tiền BHXH nên hết thời gian nghỉ thai sản tôi chưa được thanh toán chế độ. Năm 2021 công ty đã nộp hết số tiền nợ BHXH và giải thể. Xin hỏi, bây giờ tôi làm đơn xin hưởng chế độ thai sản của năm 2014 có được không?"""
found_docs = qdrant.similarity_search_with_score(
    query = query,
    k = 5,
    )

In [20]:
found_docs

[(Document(page_content='Situation: Tôi đóng BHXH từ tháng 6/2011 đến tháng 12/2014, tháng 8/2014 tôi sinh con. Do công ty nợ tiền BHXH nên hết thời gian nghỉ thai sản tôi chưa được thanh toán chế độ. Năm 2021 công ty đã nộp hết số tiền nợ BHXH và giải thể.', metadata={'_id': '7d765244-f11d-4e7b-9b7a-b2e6acfdd745', '_collection_name': 'Common_Law_Database_Tinh_Huong_CSV'}),
  0.9544498),
 (Document(page_content='Situation: Tôi đóng BHXH từ tháng 4/2019. Tháng 2/2020 đến tháng 7/2020, tôi nghỉ chế độ thai sản. Tháng 8/2020, tôi đi làm trở lại và tháng 10/2020 chấm dứt hợp đồng lao động. Xin hỏi, công ty đang nợ BHXH thì có cách nào để tôi được nhận tiền thai sản khi đã nghỉ việc không?', metadata={'_id': 'af7a7b39-f5e1-42aa-9d34-878e5c1114e5', '_collection_name': 'Common_Law_Database_Tinh_Huong_CSV'}),
  0.90431654),
 (Document(page_content='Situation: Trước đây, tôi công tác\xa0tại một đơn vị,\xa0tham gia\xa0BHXH được 3 năm. Tháng 1/2014 tôi chuyển công tác sang đơn vị\xa0khác và\xa0cũ

In [21]:
for doc, score in found_docs:
    print(doc.page_content)
    print(score)
    print("\n")

Situation: Tôi đóng BHXH từ tháng 6/2011 đến tháng 12/2014, tháng 8/2014 tôi sinh con. Do công ty nợ tiền BHXH nên hết thời gian nghỉ thai sản tôi chưa được thanh toán chế độ. Năm 2021 công ty đã nộp hết số tiền nợ BHXH và giải thể.
0.9544498


Situation: Tôi đóng BHXH từ tháng 4/2019. Tháng 2/2020 đến tháng 7/2020, tôi nghỉ chế độ thai sản. Tháng 8/2020, tôi đi làm trở lại và tháng 10/2020 chấm dứt hợp đồng lao động. Xin hỏi, công ty đang nợ BHXH thì có cách nào để tôi được nhận tiền thai sản khi đã nghỉ việc không?
0.90431654


Situation: Trước đây, tôi công tác tại một đơn vị, tham gia BHXH được 3 năm. Tháng 1/2014 tôi chuyển công tác sang đơn vị khác và cũng tiếp tục đóng BHXH. Hiện tôi đang mang thai, dự sinh vào khoảng tháng 5/2014. Vậy tôi có được tính thời gian đóng BHXH ở đơn vị cũ, cộng với thời gian đóng BHXH ở đơn vị mới để hưởng chế độ thai sản không?
0.90318817


Situation: Tôi sinh con ngày 15/7/2015 nhưng công ty lại báo giảm thai sản vào tháng 8/2015 và thu tiền BHXH t